In [1]:
import time
from datetime import datetime
import requests
import asyncio
import pandas as pd
import numpy as np
import requests as req
import json
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

In [ ]:
# This study is a personal training project aimed at analyzing Morpho Markets' risks using Pendle PT tokens as collateral.
# The objective was to incorporate the use of objects and error-catching processes.

# We first collect data on Morpho Markets and Vaults. Based on their use of PT tokens as collateral, we retrieve matching Pendle Market data.
# We assess the threshold at which liquidations are no longer profitable for liquidators, assuming a mispricing between liquidatable positions' prices and the Pendle AMM.
# This opens the way for further study, with limitations and key interests highlighted in the conclusion.


In [2]:
#Object definitions
#Definition Morpho Vaults and Markets and Pendle Markets with key attributes to be called for further analysis

class MorphoVault:
    def __init__(self, name, curator, markets, supply, liquidity):
       self.name = name
       self.curator = curator
       self.markets = {}
       self.supply = supply
       self.liquidity = liquidity

    def __repr__(self):
        return f"{self.name} Vault by {self.curator}. Supply: {self.supply}. Liquidity: {self.liquidity}"

    def AssociateMarket(self, Market):
        self.markets[Market.uniqueId] = Market

    def RemoveAllMarkets(self):
        self.markets = {}

class MorphoMarket:
    def __init__(self, name, uniqueId, c_Asset, l_Asset, utilization, borrowAssets, collateralAssets, lltv, collateralPrice):
        self.name = name
        self.uniqueId = uniqueId
        self.c_Asset = c_Asset
        self.l_Asset = l_Asset
        self.utilization = utilization
        self.lltv = lltv
        self.borrowAssets = borrowAssets
        self.collateralAssets = collateralAssets
        self.collateralPrice = collateralPrice
        
    def __repr__(self):
        return f"{self.name} Morpho Market. Collateral: {self.c_Asset}. Loan: {self.l_Asset}. LLTV = {self.lltv}"

class PendleMarket:
    def __init__(self,pt_address,yt_address,lp_address,initialAnchor,rateScalar,maturity_timestamp,yield_min,yield_max,pt_price,pt_discount,yt_price):
        self.pt_address = pt_address
        self.yt_address = yt_address
        self.lp_address = lp_address
        self.initialAnchor = initialAnchor
        self.rateScalar = rateScalar
        self.maturity_timestamp = maturity_timestamp
        self.yield_min = yield_min
        self.yield_max = yield_max
        self.pt_price = pt_price
        self.pt_discount = pt_discount

    def __repr__(self):
        return f"{self.lp_address} Pendle Market. PT address: {self.pt_address}. YT address: {self.yt_address}"
        
#Dictionnary for objects with index as unique ID (vault, market, pendle)
Vaults_dict = {}
Markets_dict = {}
Pendle_dict = {}

In [3]:
#Function definitions

def yearsToExpiry(maturity):
    return (maturity-time.time())/31556926

def liquidationIncentive(lltv):
    _beta = 0.3
    _M = 1.15
    LIF = min(_M, 1/(_beta * lltv + (1 - _beta)))
    return LIF

def convertToInt(x):
    if x is None:
        return 0 
    try:
        return int(x)
    except ValueError:
        return 0 

def saveResponseinFile(fileName,data_to_save):
    with open(f"{fileName}.json", "w", encoding="utf-8") as file:
        json.dump(data_to_save,file, indent=4)

def ReInitializeMarketsVaults():
    for vault in Vaults_dict:
         Vaults_dict[vault].RemoveAllMarkets()

In [5]:
#GET MORPHO MARKETS AND VAULTS FROM MORPHO API

async def requestMorpho(page) -> dict:

    transport = AIOHTTPTransport("https://blue-api.morpho.org/graphql")
    client = Client(transport=transport, fetch_schema_from_transport=True)
    
    rqst = f"""
    query CollateralAsset {{
      markets(skip:{page*100}) {{
items {{
      collateralAsset {{
        chain {{
          id
        }}
        address
        name
        symbol
        oraclePriceUsd
        priceUsd
        decimals
      }}
      loanAsset {{
        chain {{
          id
        }}
        address
        name
        symbol
        oraclePriceUsd
        priceUsd
        decimals
      }}
      oracleAddress
      collateralPrice
      lltv
      state {{
        borrowAssetsUsd
        borrowAssets
        borrowShares
        collateralAssets
        collateralAssetsUsd
        liquidityAssets
        liquidityAssetsUsd
        utilization
        supplyAssetsUsd
        supplyAssets
      }}
      supplyingVaults {{
        address
        symbol
        name
        asset {{
          name
          address
        }}
        state {{
          curator
          totalAssets
          totalAssetsUsd
        }}
        allocators {{
          address
        }}
        liquidity {{
          underlying
          usd
        }}
        metadata {{
          curators {{
            name
            verified
          }}
        }}
      }}
      reallocatableLiquidityAssets
      publicAllocatorSharedLiquidity {{
        assets
        id
      }}
      id
    }}
  }}
    }}"""
    
    rqst = gql(rqst)
    
    try:
        result = await client.execute_async(rqst)
        saveResponseinFile(f"MorphoVaultsandMarkets-{page}",result)
        print("Updated data from Morpho API")
    except:
        #If error during loading the data, use the previous saved file
        with open(f"MorphoVaultsandMarkets-{page}.json", "r", encoding="utf-8") as file:
            result = json.load(file)
        print("Loaded last saved file")
    finally:
        print(f"Done for [{page}]")
    return result

In [6]:
#CREATE MORPHO MARKETS AND VAULTS FROM MORPHO API RESULT

def Manage_Morpho_Answer(result):
    for market in result["markets"]["items"]:
        if market["collateralAsset"]:
            uniqueId = market["id"]
            c_Asset = market["collateralAsset"]["address"]
            c_Asset_symbol = market["collateralAsset"]["symbol"]
            l_Asset = market["loanAsset"]["address"]
            l_Asset_symbol = market["loanAsset"]["symbol"]
            name = f"{c_Asset_symbol}/{l_Asset_symbol}"
            utilization = market["state"]["utilization"]
            try:
                borrowAssets = convertToInt(market["state"]["borrowAssets"])/(10**convertToInt(market["loanAsset"]["decimals"])) # Quantity of assets
            except: 
                borrowAssets = 0
            try: 
                collateralAssets = convertToInt(market["state"]["collateralAssets"])/(10**convertToInt(market["collateralAsset"]["decimals"])) # Quantity of assets
            except:
                collateralAssets = 0
            try: 
                lltv = convertToInt(market["lltv"])/10**18
            except: 
                lltv = 0
            try: 
                collateralPrice = convertToInt(market["collateralPrice"])
            except: 
                collateralPrice = 0
            
            if not uniqueId in Markets_dict:
                Markets_dict[uniqueId] = MorphoMarket(name, uniqueId, c_Asset,l_Asset,utilization, borrowAssets, collateralAssets, lltv, collateralPrice)
    
            if len(market["supplyingVaults"]) > 0:
                for vault in market["supplyingVaults"]:
                    name = vault["name"]
                    curators_list = list()
                    metadata = vault.get("metadata", {})
                    if isinstance(metadata, dict):
                        curators = metadata.get("curators")
                        if isinstance(curators, list):
                            for curator in curators:
                                curators_list.append(curator["name"])
                    curator = curators_list
                    supply = vault["state"]["totalAssets"]
                    liquidity = vault["liquidity"]["underlying"]
                    if not name in Vaults_dict:
                        Vaults_dict[name] = MorphoVault(name, curator, {},supply, liquidity)
                    Vaults_dict[name].AssociateMarket(Markets_dict[uniqueId])
                    #As Markets are all reinitiliazed, we associate each market
                

In [7]:
#Manage Morpho API pagination

async def GetAllMorphoMarketsVaults():
    ReInitializeMarketsVaults()
    last_file = {}
    result = {}
    i=0
    while (result := await requestMorpho(i)) != last_file:
        last_file = result
        i = i+1
        if i>19:
            break
        print(f"Extracting page {i}")
        Manage_Morpho_Answer(result)
        time.sleep(2)

In [8]:
await GetAllMorphoMarketsVaults()
 #DIfferents tests succeeded in catching errors; for example with an incorrect Morpho API url 

C:\Users\alpha\anaconda3\Lib\site-packages\gql\transport\aiohttp.py:92: UserWarning: WARNING: By default, AIOHTTPTransport does not verify ssl certificates. This will be fixed in the next major version. You can set ssl=True to force the ssl certificate verification or ssl=False to disable this warning
  warnings.warn(


Updated data from Morpho API
Done for [0]
Extracting page 1
Updated data from Morpho API
Done for [1]
Extracting page 2
Updated data from Morpho API
Done for [2]
Extracting page 3
Updated data from Morpho API
Done for [3]
Extracting page 4
Updated data from Morpho API
Done for [4]
Extracting page 5
Updated data from Morpho API
Done for [5]
Extracting page 6
Updated data from Morpho API
Done for [6]
Extracting page 7
Updated data from Morpho API
Done for [7]
Extracting page 8
Updated data from Morpho API
Done for [8]


In [9]:
#Get all Pendle Markets with Pendle API
pendle_api_rqst = requests.get("https://api-v2.pendle.finance/core/v1/1/markets/active") #For Ethereum Mainnet only
all_pendle_markets = pendle_api_rqst.json()

In [10]:
#Get the Market details through Pendle front API
async def Get_Detailed_Market_Data(address) -> dict:
    pendle_market_request = requests.get("https://api-v2.pendle.finance/bff/v3/markets/all?chainId=1&q="+address+"&select=all") #For Ethereum Mainnet only
    return pendle_market_request.json()

In [11]:
#Filtering Pendle Markets used in Morpho.
#Combination Morpho and Pendle Markets to use for the analysis later

MorphoMarket_with_Pendle = []
PendleMarkets_used = []

for Pd_market in all_pendle_markets["markets"]:
    for market in Markets_dict:
        if Markets_dict[market].c_Asset.lower() == Pd_market["pt"].replace("1-",""):
            asset_type = "Collateral"
        elif Markets_dict[market].l_Asset.lower() == Pd_market["pt"].replace("1-",""):
            asset_type = "Loan"
        else:
            continue
        PendleMarkets_used.append(Pd_market)
        MarketInfo = await Get_Detailed_Market_Data(Pd_market["address"])
        Pendle_dict[Pd_market["name"]] = PendleMarket(Pd_market["pt"].replace("1-",""), 
                                                   Pd_market["yt"].replace("1-",""),
                                                   Pd_market["address"], 
                                                   MarketInfo["initialAnchorList"][0],
                                                   MarketInfo["scalarRootList"][0],
                                                   MarketInfo["expiryList"][0],
                                                   MarketInfo["extendedInfoList"][0]["yieldRange"]["min"],
                                                   MarketInfo["extendedInfoList"][0]["yieldRange"]["max"],
                                                   1 / float(MarketInfo["marketMathDataList"][0]["ptExchangeRate"]),
                                                   MarketInfo["ptDiscountList"][0], 
                                                   MarketInfo["marketMathDataList"][0]["interestFeeRate"])
        MorphoMarket_with_Pendle.append({"MorphoMarket_Asset":asset_type,"MorphoMarket":market,"Morpho_lltv": Markets_dict[market].lltv, "Morpho_Collateral":Markets_dict[market].c_Asset.lower(),
                                         "Morpho_Loan":Markets_dict[market].l_Asset.lower(),"Morpho_borrowAssets":Markets_dict[market].borrowAssets, 
                                         "Morpho_collateralAssets":Markets_dict[market].collateralAssets, "Morpho_collateralPrice":Markets_dict[market].collateralPrice,
                                         "Pendle_Market":Pd_market["name"], "Pendle_PT":Pd_market["pt"].replace("1-",""), "Pendle_YT":Pd_market["yt"].replace("1-",""),
                                        "Pendle_Address":Pd_market["address"], "Pendle_Anchor": MarketInfo["initialAnchorList"][0],
                                         "Pendle_scalarRoot":MarketInfo["scalarRootList"][0],"Pendle_expiry":MarketInfo["expiryList"][0],
                                         "Pendle_yieldMin":MarketInfo["extendedInfoList"][0]["yieldRange"]["min"],"Pendle_yieldMax":MarketInfo["extendedInfoList"][0]["yieldRange"]["max"],
                                         "Pendle_PT_price":1 / float(MarketInfo["marketMathDataList"][0]["ptExchangeRate"]),"Pendle_PT_discount":MarketInfo["ptDiscountList"][0],
                                         "Pendle_FeeRoot":MarketInfo["marketMathDataList"][0]["interestFeeRate"]})



In [12]:
Markets_combined_Pdl_Mrp = pd.DataFrame(MorphoMarket_with_Pendle)
#print(Markets_combined_Pdl_Mrp)

In [100]:
#ANALYSIS OF PENDLE MARKETS RISKS
#We define the incentive earned by liquidator and calculate IR_incent_delta = an approximation of the implied interest rate change that would cover the full liquidation incentive and make it 0.

Markets_combined_Pdl_Mrp["Incentive"] = Markets_combined_Pdl_Mrp["Morpho_lltv"].apply(liquidationIncentive)
Markets_combined_Pdl_Mrp["IR_incent_delta"] = (1/(1-(Markets_combined_Pdl_Mrp["Incentive"]-1)))**(1/yearsToExpiry(Markets_combined_Pdl_Mrp["Pendle_expiry"]))-1

#We then analyze the difference between the implied rate to cover full liquidation from the actual rate and the maximum yield of the configured Pendle Market. This is an approximation
Markets_combined_Pdl_Mrp["Current_PT_yield"] = (1/(1-Markets_combined_Pdl_Mrp["Pendle_PT_discount"]))**(1/yearsToExpiry(Markets_combined_Pdl_Mrp["Pendle_expiry"]))-1
Markets_combined_Pdl_Mrp["UpperYield_Liquid_No_Profit"] = Markets_combined_Pdl_Mrp["Current_PT_yield"] + Markets_combined_Pdl_Mrp["IR_incent_delta"] 
Markets_combined_Pdl_Mrp["No_Profit_Possible_By_Market"] = Markets_combined_Pdl_Mrp["UpperYield_Liquid_No_Profit"] < Markets_combined_Pdl_Mrp["Pendle_yieldMax"]

#It represents the risk of a liquidator being unable to benefit from their liquidation incentive due to mispricing between the Morpho Oracle price and the Pendle Market.
#The amount seizable for liquidators would not be sufficient to cover the price deviation.
#For markets at risk, we calculate the price delta until it reaches the upper bound of the liquidity pool, to assess the maximum loss incurred by trading at the pool’s maximum range without being able to liquidate.
Markets_at_risk = Markets_at_risk[Markets_at_risk["No_Profit_Possible_By_Market"] == True]
Markets_at_risk["Minprice"] =  1/((1+Markets_at_risk["Pendle_yieldMax"])**(yearsToExpiry(Markets_at_risk["Pendle_expiry"])))
Markets_at_risk["Price_Impact"] = Markets_at_risk["Minprice"]- (Markets_at_risk["Pendle_PT_price"]/(Markets_at_risk["Incentive"]))

In [112]:
print("Following Pendle markets allow for trading in AMM in ranges where a current mispricing with Morpho Oracle might result in unprofitable liquidations")
print(Markets_at_risk)

Following Pendle markets allow for trading in AMM in ranges where a current mispricing with Morpho Oracle might result in unprofitable liquidations
   MorphoMarket_Asset                          MorphoMarket  Morpho_lltv  \
18         Collateral  c094041f-1387-4688-a0e9-aed7302af29b        0.915   
19         Collateral  4461471f-aa51-4175-8632-a724228045bd        0.915   
20         Collateral  f74192df-8957-4d2b-ba4c-d3ecdc3dc50b        0.915   
21         Collateral  297f6d14-4956-433c-b651-1c0c75dc2853        0.915   
24         Collateral  4b660fc8-eff1-4027-98de-0f6aedd9af84        0.915   
25         Collateral  525ae7e9-efb6-4975-a673-3d9865c5fb8f        0.915   
26         Collateral  044a165d-633f-4205-bea9-47508e244b98        0.915   
27         Collateral  df45f11c-1d28-4aa2-b444-1503f6daa98f        0.915   
28         Collateral  28023d7a-e930-431f-9edb-6ab151984569        0.915   
29         Collateral  99f05eec-aa2e-4efe-b617-330887a81144        0.915   

               

In [110]:
#Subset of markets with higher risks

Markets_with_risk = Markets_at_risk["Pendle_Market"][Markets_at_risk["Price_Impact"]<-0.005].unique()
#print(Markets_with_risk)
print(f"For a loan near the LLTV today, a mispricing between Morpho Oracle and AMM could make liquidation unprofitable below the threshold of -0.5% in the following markets: {Markets_with_risk}")

['tETH' 'eUSDe']
For a loan near the LLTV today, a mispricing between Morpho Oracl and AMM could make liquidation unprofitable above the threshold of 0,5% in the following markets: ['tETH' 'eUSDe']


In [ ]:
#Next steps to implement: 
#1. Estimate the amount needed to reach Upper Yield limit in Pendle Pool by using https://api-v2.pendle.finance/core/docs#/Markets/MarketsController_getSwapAmountToChangeImpliedApy
#2. Gathering information about PT liquidity holder and borrowers and their holding proportion compared with the amount needed to move Pendle Market

#Next analysis
#3. Compare the Morpho Oracle Price with the actual Pendle price.
#4. Asssess again scenarios where liquidation or bad debt could occur and identify the affected markets
#5. Evaluate Vault's ability to withdraw supplied liquidity to markets when risks materialize

#Limitations 
# Currently applicable only on the Ethereum Blockchain (Pendle API requests only the markets on blockchain ID:1)
# Calculations are approximated
